# MLOps

parquet file ->(raw data) -> Data Preparation ->(features),(targets) -> Model Training -> (artifact) -> ML model

# Feature Store

hopswork.ai

generate API key to use the Python SDK to communicate with the feature store